In [4]:
!pip install textstat
!pip install tensorflow
!pip install keras
!pip install datasets
!pip install transformers

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys
import nltk
from nltk.stem.porter import *
import string
import re
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Flatten, Activation, Embedding, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from torch.autograd import Variable
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import random

path_to_file = "/content/drive/MyDrive/nlpdata/hatespeech_data_clean_text_for_bert.csv"

data = pd.read_csv(path_to_file,encoding='latin-1')

train, test= train_test_split(data,test_size=0.2, random_state=25)

train.to_csv('train.csv')
test.to_csv('test.csv')
data = load_dataset('csv',data_files={'train': 'train.csv', 'test': 'test.csv'})

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-f5255563b4b049da/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from gensim.utils import pickle

# Creating a embedding matrix
load_pretrained = False
embedding_dimension = 100

if load_pretrained:
    with open('content/drive/MyDrive/braer/nlpdata/word2vec_model_local', 'rb') as file_in:
        model = pickle.load(file_in)
else:
    model = Word2Vec([tweet.split() for tweet in data["train"]["tweet"]], min_count = 2, size = embedding_dimension,
                     window = 10, sg = 1)
    with open('content/drive/MyDrive/braer/nlpdata/word2vec_model_local', 'wb') as file_out:
        pickle.dump(model, file_out)

# tokenize each tweet in dataset
max_vocab_size = len(model.wv.vocab)
tokenizer = Tokenizer(num_words = max_vocab_size)
tokenizer.fit_on_texts(data["train"]["tweet"])


In [ ]:
# create embedding matrix where each row corresponds to a vector for a word
embedding_matrix = np.zeros([max_vocab_size + 1, embedding_dimension])
for word, index in tokenizer.word_index.items():
    if word in model.wv.vocab and index < max_vocab_size:
        vector = model[word]
        embedding_matrix[index,:] = vector

In [ ]:
# convert tweets to index sequence vectors
X_train_nn = tokenizer.texts_to_sequences(data["train"]["tweet"])
X_test_nn = tokenizer.texts_to_sequences(data["test"]["tweet"])
max_sequence_length = np.array([len(sequence) for sequence in X_train_nn]).max()

# pad index sequence vectors so they all have same length
X_train_nn = pad_sequences(X_train_nn, maxlen = max_sequence_length, padding = 'post', truncating = 'post')
X_test_nn = pad_sequences(X_test_nn, maxlen = max_sequence_length, padding = 'post', truncating = 'post')

# create hate speech label matrix y
y_train_nn = to_categorical(data["train"]["hate_speech"])
y_test_nn = to_categorical(data["train"]["hate_speech"])

In [ ]:
# Initial setup of multilayer perceptron model
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],
                 embedding_matrix.shape[1],
                 weights = [embedding_matrix],
                 input_shape = (X_train_nn.shape[1],)))
model.add(Flatten())
model.add(Dense(75, activation = 'relu'))
model.add(Dropout(0.7))
model.add(Dense(20, activation = 'relu'))
model.add(Dropout(0.7))
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Testing this out
early_stopping = EarlyStopping(patience = 2, monitor = 'train_acc')
report = model.fit(X_train_nn, y_train_nn, validation_split = 0.3, epochs = 14, callbacks = [early_stopping], batch_size = 200, verbose = False)

In [1]:
from sklearn.metrics import accuracy_score, f1_score

def prob_to_prediction(predict_prob):
    predictions = np.array([np.argmax(row) for row in predict_prob])
    return predictions

def evaluate(y, y_pred):
    print('Accuracy:', accuracy_score(y, y_pred))
    print('F1 Score: ', f1_score(y, y_pred, average = 'weighted'))